# 🤖 Nong-View AI 농업영상분석 플랫폼 - 완전 통합 노트북 v2.0

**프로젝트 현황**: 92% → 98% 완성 목표  
**핵심 업데이트**: POD3(타일링), POD5(병합), POD6(GPKG 내보내기) 통합  
**개발 날짜**: 2025-10-27  
**버전**: v2.0

## 📋 v2.0 신규 기능
- ✅ v1.0 모든 기능 포함
- 🆕 POD3: 대용량 이미지 타일링 시스템
- 🆕 POD5: 중복 검출 결과 병합 엔진
- 🆕 POD6: 표준 GPKG 파일 내보내기
- 🆕 Crops API 완전 구현
- 🆕 Exports API 완전 구현
- 🆕 Statistics API 구현


## 🔧 1. 환경 설정 및 의존성 설치

In [ ]:
# 필수 패키지 설치 (v2.0 추가 패키지 포함)
!pip install fastapi uvicorn sqlalchemy alembic psycopg2-binary redis
!pip install pydantic[email] python-multipart
!pip install rasterio geopandas shapely fiona
!pip install ultralytics torch torchvision
!pip install pillow opencv-python numpy pandas
!pip install python-jose[cryptography] passlib[bcrypt]
!pip install pytest pytest-asyncio httpx
!pip install rtree tqdm  # POD3, POD5용 추가 패키지

In [ ]:
# 기본 라이브러리 임포트
import os
import sys
import json
import logging
import asyncio
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any, Union, Tuple, Generator
from pathlib import Path
from uuid import UUID, uuid4
import hashlib
import time
import sqlite3

# 데이터 처리
import numpy as np
import pandas as pd
from PIL import Image
import cv2
from tqdm import tqdm

# 지리정보 처리
import rasterio
from rasterio.windows import Window
from rasterio.transform import from_bounds
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, box, mapping
from shapely.ops import unary_union
from rasterio.crs import CRS
from rtree import index

# 웹 프레임워크
from fastapi import FastAPI, HTTPException, Depends, UploadFile, File, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, FileResponse
from pydantic import BaseModel, Field, validator

# 데이터베이스
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Float, Text, Boolean, ForeignKey, JSON
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session, relationship
from sqlalchemy.dialects.postgresql import UUID as PG_UUID

# AI/ML
from ultralytics import YOLO
import torch
from concurrent.futures import ThreadPoolExecutor

# 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ 모든 라이브러리 임포트 완료 (v2.0)")

## ⚙️ 2. 설정 및 구성

In [ ]:
# 설정 클래스 정의 (v2.0 확장)
class Settings:
    """애플리케이션 설정 v2.0"""
    PROJECT_NAME: str = "Nong-View API"
    VERSION: str = "2.0.0"
    API_V1_STR: str = "/api/v1"
    ENVIRONMENT: str = "development"
    DEBUG: bool = True
    
    # 서버 설정
    HOST: str = "127.0.0.1"
    PORT: int = 8000
    
    # 데이터베이스 설정
    DATABASE_URL: str = "sqlite:///./nongview_v2.db"
    
    # 파일 저장 설정
    UPLOAD_PATH: str = "./uploads"
    CROP_PATH: str = "./crops"
    TILE_PATH: str = "./tiles"  # v2.0 추가
    EXPORT_PATH: str = "./exports"
    MAX_FILE_SIZE: int = 2 * 1024 * 1024 * 1024  # 2GB
    
    # AI 모델 설정
    MODEL_PATH: str = "./models"
    MAX_WORKERS: int = 4
    
    # 타일링 설정 (v2.0 추가)
    TILE_SIZE: int = 640
    TILE_OVERLAP: float = 0.2
    
    # 병합 설정 (v2.0 추가)
    IOU_THRESHOLD: float = 0.5
    CONFIDENCE_THRESHOLD: float = 0.3
    
    # GPKG 내보내기 설정 (v2.0 추가)
    OUTPUT_CRS: str = "EPSG:5186"  # UTM-K
    PRIVACY_MASK: bool = True
    
    # 보안 설정
    SECRET_KEY: str = "dev-secret-key-change-in-production"
    ALGORITHM: str = "HS256"
    ACCESS_TOKEN_EXPIRE_MINUTES: int = 1440

settings = Settings()

# 디렉토리 생성
for path in [settings.UPLOAD_PATH, settings.CROP_PATH, settings.TILE_PATH, 
             settings.EXPORT_PATH, settings.MODEL_PATH]:
    Path(path).mkdir(parents=True, exist_ok=True)

print("✅ 설정 완료 (v2.0)")
print(f"📁 업로드 경로: {settings.UPLOAD_PATH}")
print(f"🔲 타일 경로: {settings.TILE_PATH}")
print(f"📦 내보내기 경로: {settings.EXPORT_PATH}")
print(f"🗄️ 데이터베이스: {settings.DATABASE_URL}")

## 🗄️ 3. 데이터베이스 모델 정의 (v2.0 확장)

In [ ]:
# SQLAlchemy 설정
engine = create_engine(
    settings.DATABASE_URL,
    connect_args={"check_same_thread": False} if "sqlite" in settings.DATABASE_URL else {}
)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# 데이터베이스 모델 정의 (v2.0 확장)
class User(Base):
    """사용자 모델"""
    __tablename__ = "users"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    username = Column(String(50), unique=True, index=True, nullable=False)
    email = Column(String(100), unique=True, index=True, nullable=False)
    hashed_password = Column(String(100), nullable=False)
    is_active = Column(Boolean, default=True)
    is_superuser = Column(Boolean, default=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    # 관계
    images = relationship("Image", back_populates="owner")
    analyses = relationship("Analysis", back_populates="owner")


class Image(Base):
    """이미지 모델"""
    __tablename__ = "images"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    filename = Column(String(255), nullable=False)
    file_path = Column(String(500), nullable=False)
    file_size = Column(Integer, nullable=False)
    format = Column(String(20), nullable=False)
    status = Column(String(20), default="uploading")
    
    # 메타데이터
    description = Column(Text)
    region_name = Column(String(100))
    capture_date = Column(DateTime)
    drone_model = Column(String(100))
    camera_model = Column(String(100))
    altitude = Column(Float)
    overlap = Column(Float)
    tags = Column(JSON)  # List[str]
    
    # 지리 정보
    crs = Column(String(50))
    bounds = Column(JSON)  # {"minx": float, "miny": float, "maxx": float, "maxy": float}
    resolution = Column(Float)
    width = Column(Integer)
    height = Column(Integer)
    bands = Column(Integer)
    
    # 소유자 및 시간
    owner_id = Column(String, ForeignKey("users.id"), nullable=False)
    uploaded_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    # 관계
    owner = relationship("User", back_populates="images")
    analyses = relationship("Analysis", back_populates="image")
    crop_results = relationship("CropResult", back_populates="image")
    tiles = relationship("Tile", back_populates="image")  # v2.0 추가


class Analysis(Base):
    """분석 작업 모델"""
    __tablename__ = "analyses"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    name = Column(String(200), nullable=False)
    description = Column(Text)
    status = Column(String(20), default="pending")  # pending, running, completed, error
    
    # 분석 설정
    analysis_type = Column(String(50), nullable=False)  # crop, facility, landuse
    model_version = Column(String(50))
    confidence_threshold = Column(Float, default=0.5)
    use_tiling = Column(Boolean, default=False)  # v2.0 추가
    config = Column(JSON)  # 분석 설정
    
    # 결과
    total_detections = Column(Integer, default=0)
    total_tiles = Column(Integer, default=0)  # v2.0 추가
    processing_time = Column(Float)
    error_message = Column(Text)
    result_data = Column(JSON)  # 분석 결과 요약
    
    # 관계
    image_id = Column(String, ForeignKey("images.id"), nullable=False)
    owner_id = Column(String, ForeignKey("users.id"), nullable=False)
    
    # 시간
    created_at = Column(DateTime, default=datetime.utcnow)
    started_at = Column(DateTime)
    completed_at = Column(DateTime)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    # 관계
    image = relationship("Image", back_populates="analyses")
    owner = relationship("User", back_populates="analyses")
    crop_results = relationship("CropResult", back_populates="analysis")
    exports = relationship("Export", back_populates="analysis")
    tiles = relationship("Tile", back_populates="analysis")  # v2.0 추가


class Tile(Base):  # v2.0 신규 추가
    """타일 모델"""
    __tablename__ = "tiles"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    tile_path = Column(String(500), nullable=False)
    tile_size = Column(Integer, nullable=False)
    
    # 위치 정보
    row = Column(Integer, nullable=False)
    col = Column(Integer, nullable=False)
    x_offset = Column(Integer, nullable=False)
    y_offset = Column(Integer, nullable=False)
    
    # 좌표 정보
    bounds = Column(JSON)  # 지리적 좌표
    pixel_bounds = Column(JSON)  # 픽셀 좌표
    
    # AI 분석 결과
    detection_count = Column(Integer, default=0)
    detection_data = Column(JSON)  # 검출 결과
    processed = Column(Boolean, default=False)
    
    # 관계
    image_id = Column(String, ForeignKey("images.id"), nullable=False)
    analysis_id = Column(String, ForeignKey("analyses.id"))
    
    # 시간
    created_at = Column(DateTime, default=datetime.utcnow)
    processed_at = Column(DateTime)
    
    # 관계
    image = relationship("Image", back_populates="tiles")
    analysis = relationship("Analysis", back_populates="tiles")


class CropResult(Base):
    """크로핑 결과 모델"""
    __tablename__ = "crop_results"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    pnu = Column(String(19), nullable=False, index=True)  # 필지고유번호
    crop_path = Column(String(500), nullable=False)
    crop_size = Column(Integer)
    
    # 지오메트리 정보
    geometry_type = Column(String(50))
    area = Column(Float)
    perimeter = Column(Float)
    bounds = Column(JSON)
    
    # AI 분석 결과
    detected_class = Column(String(50))
    confidence = Column(Float)
    detection_count = Column(Integer, default=0)
    detection_data = Column(JSON)  # 상세 검출 결과
    merged_from_tiles = Column(Boolean, default=False)  # v2.0 추가
    
    # 관계
    image_id = Column(String, ForeignKey("images.id"), nullable=False)
    analysis_id = Column(String, ForeignKey("analyses.id"))
    
    # 시간
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    # 관계
    image = relationship("Image", back_populates="crop_results")
    analysis = relationship("Analysis", back_populates="crop_results")


class Export(Base):
    """내보내기 작업 모델"""
    __tablename__ = "exports"
    
    id = Column(String, primary_key=True, default=lambda: str(uuid4()))
    name = Column(String(200), nullable=False)
    export_type = Column(String(50), nullable=False)  # gpkg, geojson, csv, shapefile
    status = Column(String(20), default="pending")  # pending, processing, completed, error
    
    # 파일 정보
    file_path = Column(String(500))
    file_size = Column(Integer)
    download_count = Column(Integer, default=0)
    
    # 내보내기 설정
    include_geometry = Column(Boolean, default=True)
    include_metadata = Column(Boolean, default=True)
    privacy_applied = Column(Boolean, default=False)  # v2.0 추가
    layer_count = Column(Integer, default=1)  # v2.0 추가
    filter_config = Column(JSON)  # 필터 설정
    
    # 오류 정보
    error_message = Column(Text)
    
    # 관계
    analysis_id = Column(String, ForeignKey("analyses.id"), nullable=False)
    
    # 시간
    created_at = Column(DateTime, default=datetime.utcnow)
    completed_at = Column(DateTime)
    expires_at = Column(DateTime)  # 다운로드 만료일
    
    # 관계
    analysis = relationship("Analysis", back_populates="exports")


# 데이터베이스 테이블 생성
Base.metadata.create_all(bind=engine)

print("✅ 데이터베이스 모델 생성 완료 (v2.0)")
print(f"📊 생성된 테이블: {list(Base.metadata.tables.keys())}")

## 🔗 4. 데이터베이스 연결 관리

In [ ]:
# 데이터베이스 세션 의존성
def get_db():
    """데이터베이스 세션 생성"""
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# 테스트용 사용자 생성
def create_test_user(db: Session):
    """테스트용 사용자 생성"""
    existing_user = db.query(User).filter(User.username == "testuser").first()
    if not existing_user:
        test_user = User(
            username="testuser",
            email="test@example.com",
            hashed_password="hashed_password_here",
            is_active=True
        )
        db.add(test_user)
        db.commit()
        db.refresh(test_user)
        print(f"✅ 테스트 사용자 생성: {test_user.id}")
        return test_user
    else:
        print(f"ℹ️ 테스트 사용자 존재: {existing_user.id}")
        return existing_user

# 테스트 사용자 생성
with SessionLocal() as db:
    test_user = create_test_user(db)
    
print("✅ 데이터베이스 연결 관리 설정 완료")

## 🔌 5. POD 모듈 통합 (v2.0 - 전체 모듈)
### POD1: 데이터 수집 및 관리

In [ ]:
# POD1: 데이터 수집 및 관리 모듈 (v1.0과 동일)
class DataRegistry:
    """중앙 데이터 레지스트리"""
    
    def __init__(self, storage_path: str):
        self.storage_path = Path(storage_path)
        self.storage_path.mkdir(parents=True, exist_ok=True)
    
    def extract_image_metadata(self, file_path: Path) -> dict:
        """이미지 메타데이터 추출"""
        try:
            with rasterio.open(file_path) as src:
                bounds = {
                    "minx": src.bounds.left,
                    "miny": src.bounds.bottom,
                    "maxx": src.bounds.right,
                    "maxy": src.bounds.top
                }
                
                # 해상도 계산
                res_x = (src.bounds.right - src.bounds.left) / src.width
                res_y = (src.bounds.top - src.bounds.bottom) / src.height
                resolution = (res_x + res_y) / 2
                
                metadata = {
                    "crs": str(src.crs) if src.crs else "EPSG:5186",
                    "resolution": resolution,
                    "bounds": bounds,
                    "width": src.width,
                    "height": src.height,
                    "bands": src.count,
                    "format": src.driver
                }
                
                return metadata
                
        except Exception as e:
            logger.error(f"메타데이터 추출 실패: {e}")
            raise
    
    def save_file(self, upload_file: UploadFile) -> tuple[str, int]:
        """파일 저장"""
        file_id = str(uuid4())
        file_extension = Path(upload_file.filename).suffix
        file_path = self.storage_path / f"{file_id}{file_extension}"
        
        # 파일 저장
        content = upload_file.file.read()
        with open(file_path, "wb") as f:
            f.write(content)
        
        return str(file_path), len(content)

# 전역 데이터 레지스트리 인스턴스
data_registry = DataRegistry(settings.UPLOAD_PATH)

print("✅ POD1 데이터 수집 모듈 통합 완료")

### POD2: 크로핑 엔진

In [ ]:
# POD2: 크로핑 엔진 (v1.0과 동일)
class CroppingEngine:
    """ROI 추출 및 크로핑 엔진"""
    
    def __init__(self, output_dir: str):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
    
    def crop_by_shapes(self, image_path: str, shapefile_path: str) -> list:
        """Shapefile 기반 이미지 크로핑"""
        try:
            # Shapefile 로드
            gdf = gpd.read_file(shapefile_path)
            
            results = []
            
            with rasterio.open(image_path) as src:
                for idx, row in gdf.iterrows():
                    pnu = row.get('PNU', f'unknown_{idx}').zfill(19)
                    geometry = row.geometry
                    
                    # 크로핑 수행
                    try:
                        # 지오메트리로 마스킹
                        out_image, out_transform = rasterio.mask.mask(
                            src, [geometry], crop=True
                        )
                        
                        # 출력 파일 경로
                        crop_id = str(uuid4())
                        crop_filename = f"crop_{crop_id}_{pnu}.tif"
                        crop_path = self.output_dir / crop_filename
                        
                        # 크롭된 이미지 저장
                        out_meta = src.meta
                        out_meta.update({
                            "driver": "GTiff",
                            "height": out_image.shape[1],
                            "width": out_image.shape[2],
                            "transform": out_transform
                        })
                        
                        with rasterio.open(crop_path, "w", **out_meta) as dest:
                            dest.write(out_image)
                        
                        # 결과 정보
                        result = {
                            "crop_id": crop_id,
                            "pnu": pnu,
                            "crop_path": str(crop_path),
                            "crop_size": crop_path.stat().st_size,
                            "geometry_type": geometry.geom_type,
                            "area": geometry.area,
                            "perimeter": geometry.length,
                            "bounds": {
                                "minx": geometry.bounds[0],
                                "miny": geometry.bounds[1],
                                "maxx": geometry.bounds[2],
                                "maxy": geometry.bounds[3]
                            }
                        }
                        
                        results.append(result)
                        logger.info(f"크로핑 완료: {pnu}")
                        
                    except Exception as e:
                        logger.error(f"크로핑 실패 - PNU {pnu}: {e}")
                        continue
            
            return results
            
        except Exception as e:
            logger.error(f"크로핑 엔진 오류: {e}")
            raise

# 전역 크로핑 엔진 인스턴스
cropping_engine = CroppingEngine(settings.CROP_PATH)

print("✅ POD2 크로핑 엔진 통합 완료")

### POD3: 타일링 시스템 (v2.0 신규 추가)

In [ ]:
# POD3: 타일링 시스템 (v2.0 신규)
class TilingEngine:
    """대용량 이미지 타일링 엔진"""
    
    def __init__(self, tile_size: int = None, overlap: float = None, output_dir: str = None):
        self.tile_size = tile_size or settings.TILE_SIZE
        self.overlap = overlap or settings.TILE_OVERLAP
        self.output_dir = Path(output_dir or settings.TILE_PATH)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.stride = int(self.tile_size * (1 - self.overlap))
    
    def calculate_grid_size(self, image_width: int, image_height: int) -> tuple[int, int]:
        """그리드 크기 계산"""
        cols = (image_width - self.tile_size) // self.stride + 1
        rows = (image_height - self.tile_size) // self.stride + 1
        
        # 나머지 부분이 있으면 추가 타일 생성
        if (image_width - self.tile_size) % self.stride > 0:
            cols += 1
        if (image_height - self.tile_size) % self.stride > 0:
            rows += 1
            
        return rows, cols
    
    def generate_tiles(self, image_path: str, analysis_id: str) -> list:
        """이미지를 타일로 분할"""
        try:
            with rasterio.open(image_path) as src:
                image_width = src.width
                image_height = src.height
                transform = src.transform
                crs = src.crs
                
                rows, cols = self.calculate_grid_size(image_width, image_height)
                
                logger.info(f"타일링 시작: {image_path} -> {rows}x{cols} = {rows*cols}개 타일")
                
                tiles = []
                
                for row in tqdm(range(rows), desc="타일링 진행"):
                    for col in range(cols):
                        # 픽셀 좌표 계산
                        x_start = col * self.stride
                        y_start = row * self.stride
                        x_end = min(x_start + self.tile_size, image_width)
                        y_end = min(y_start + self.tile_size, image_height)
                        
                        # 실제 타일 크기
                        actual_width = x_end - x_start
                        actual_height = y_end - y_start
                        
                        # 너무 작은 타일은 제외
                        if actual_width < self.tile_size * 0.5 or actual_height < self.tile_size * 0.5:
                            continue
                        
                        # 윈도우 생성
                        window = Window(x_start, y_start, actual_width, actual_height)
                        
                        # 지리적 좌표 계산
                        bounds = rasterio.windows.bounds(window, transform)
                        
                        # 타일 ID 생성
                        tile_id = str(uuid4())
                        tile_filename = f"tile_{tile_id}_{row}_{col}.tif"
                        tile_path = self.output_dir / analysis_id / tile_filename
                        tile_path.parent.mkdir(parents=True, exist_ok=True)
                        
                        # 타일 이미지 저장
                        tile_data = src.read(window=window)
                        tile_transform = rasterio.windows.transform(window, transform)
                        
                        tile_meta = src.meta.copy()
                        tile_meta.update({
                            'height': actual_height,
                            'width': actual_width,
                            'transform': tile_transform
                        })
                        
                        with rasterio.open(tile_path, 'w', **tile_meta) as dst:
                            dst.write(tile_data)
                        
                        # 타일 정보 생성
                        tile_info = {
                            "tile_id": tile_id,
                            "tile_path": str(tile_path),
                            "row": row,
                            "col": col,
                            "x_offset": x_start,
                            "y_offset": y_start,
                            "bounds": {
                                "minx": bounds.left,
                                "miny": bounds.bottom,
                                "maxx": bounds.right,
                                "maxy": bounds.top
                            },
                            "pixel_bounds": {
                                "x_start": x_start,
                                "y_start": y_start,
                                "x_end": x_end,
                                "y_end": y_end
                            },
                            "tile_size": actual_width * actual_height
                        }
                        
                        tiles.append(tile_info)
                
                logger.info(f"타일링 완료: {len(tiles)}개 타일 생성")
                return tiles
                
        except Exception as e:
            logger.error(f"타일링 실패: {e}")
            raise

# 전역 타일링 엔진 인스턴스
tiling_engine = TilingEngine()

print("✅ POD3 타일링 엔진 통합 완료 (v2.0 신규)")
print(f"🔲 타일 크기: {tiling_engine.tile_size}px")
print(f"📐 겹침률: {tiling_engine.overlap*100}%")
print(f"📁 타일 저장소: {tiling_engine.output_dir}")

### POD4: AI 추론 엔진

In [ ]:
# POD4: AI 추론 엔진 (v1.0 기반, v2.0 확장)
class AIInferenceEngine:
    """YOLOv11 기반 AI 추론 엔진 v2.0"""
    
    def __init__(self):
        self.models = {}
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        logger.info(f"AI 엔진 초기화 - 디바이스: {self.device}")
        
        # 클래스 매핑
        self.class_mappings = {
            'crop': {
                0: 'IRG',      # 이탈리안 라이그라스
                1: 'BARLEY',   # 보리
                2: 'WHEAT',    # 밀
                3: 'CORN_SILAGE',  # 옥수수사일리지
                4: 'HAY',      # 건초
                5: 'UNKNOWN'   # 미분류
            },
            'facility': {
                0: 'GREENHOUSE_SINGLE',  # 단동 비닐하우스
                1: 'GREENHOUSE_MULTI',   # 연동 비닐하우스
                2: 'STORAGE',            # 저장시설
                3: 'LIVESTOCK',          # 축사
                4: 'SILO',              # 사일로
                5: 'UNKNOWN'            # 미분류
            }
        }
    
    def load_model(self, model_type: str, model_path: str = None):
        """YOLO 모델 로드"""
        try:
            if model_path and Path(model_path).exists():
                model = YOLO(model_path)
            else:
                # 기본 YOLOv8 모델 사용 (테스트용)
                model = YOLO('yolov8n.pt')
                logger.warning(f"사용자 모델을 찾을 수 없어 기본 모델 사용: {model_type}")
            
            self.models[model_type] = model
            logger.info(f"모델 로드 완료: {model_type}")
            
        except Exception as e:
            logger.error(f"모델 로드 실패 - {model_type}: {e}")
            raise
    
    def predict(self, image_path: str, model_type: str, confidence: float = 0.5) -> dict:
        """이미지 예측 수행"""
        try:
            if model_type not in self.models:
                raise ValueError(f"모델이 로드되지 않음: {model_type}")
            
            model = self.models[model_type]
            
            # 예측 수행
            results = model(image_path, conf=confidence, device=self.device)
            
            detections = []
            
            for result in results:
                boxes = result.boxes
                if boxes is not None:
                    for box in boxes:
                        # 바운딩 박스 좌표
                        x1, y1, x2, y2 = box.xyxy[0].tolist()
                        confidence_score = box.conf[0].item()
                        class_id = int(box.cls[0].item())
                        
                        # 클래스 이름 매핑
                        class_name = self.class_mappings.get(model_type, {}).get(
                            class_id, f"class_{class_id}"
                        )
                        
                        detection = {
                            "class_id": class_id,
                            "class_name": class_name,
                            "confidence": confidence_score,
                            "bbox": [x1, y1, x2, y2],
                            "area": (x2 - x1) * (y2 - y1)
                        }
                        
                        detections.append(detection)
            
            return {
                "detections": detections,
                "detection_count": len(detections),
                "model_type": model_type,
                "confidence_threshold": confidence
            }
            
        except Exception as e:
            logger.error(f"예측 실패: {e}")
            raise
    
    def predict_batch(self, image_paths: list, model_type: str, confidence: float = 0.5) -> list:
        """배치 예측 수행 (v2.0 확장)"""
        results = []
        
        # 병렬 처리를 위한 ThreadPoolExecutor 사용
        with ThreadPoolExecutor(max_workers=settings.MAX_WORKERS) as executor:
            # 비동기 작업 제출
            future_to_path = {
                executor.submit(self.predict, path, model_type, confidence): path 
                for path in image_paths
            }
            
            # 결과 수집
            for future in tqdm(future_to_path, desc="AI 추론 진행"):
                image_path = future_to_path[future]
                try:
                    result = future.result()
                    result['image_path'] = image_path
                    results.append(result)
                except Exception as e:
                    logger.error(f"배치 예측 실패 - {image_path}: {e}")
                    results.append({
                        'image_path': image_path,
                        'error': str(e)
                    })
        
        return results

# 전역 AI 엔진 인스턴스
ai_engine = AIInferenceEngine()

# 기본 모델 로드 (테스트용)
try:
    ai_engine.load_model('crop')
    ai_engine.load_model('facility')
    print("✅ POD4 AI 추론 엔진 통합 완료 (v2.0 확장)")
except Exception as e:
    print(f"⚠️ AI 모델 로드 실패: {e}")
    print("기본 YOLO 모델을 다운로드합니다...")

### POD5: 결과 병합 엔진 (v2.0 신규 추가)

In [ ]:
# POD5: 결과 병합 엔진 (v2.0 신규)
class MergeEngine:
    """타일 기반 검출 결과 병합 엔진"""
    
    def __init__(self, iou_threshold: float = None, confidence_threshold: float = None):
        self.iou_threshold = iou_threshold or settings.IOU_THRESHOLD
        self.confidence_threshold = confidence_threshold or settings.CONFIDENCE_THRESHOLD
    
    def calculate_iou(self, box1: list, box2: list) -> float:
        """두 바운딩 박스의 IoU 계산"""
        x1_1, y1_1, x2_1, y2_1 = box1
        x1_2, y1_2, x2_2, y2_2 = box2
        
        # 교집합 영역 계산
        x1_i = max(x1_1, x1_2)
        y1_i = max(y1_1, y1_2)
        x2_i = min(x2_1, x2_2)
        y2_i = min(y2_1, y2_2)
        
        if x2_i <= x1_i or y2_i <= y1_i:
            return 0.0
        
        intersection = (x2_i - x1_i) * (y2_i - y1_i)
        
        # 합집합 영역 계산
        area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
        area2 = (x2_2 - x1_2) * (y2_2 - y1_2)
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0.0
    
    def convert_to_global_coords(self, tile_results: list, tile_metadata: list) -> list:
        """타일 좌표를 전역 좌표로 변환"""
        global_detections = []
        
        for tile_result in tile_results:
            if 'error' in tile_result:
                continue
                
            image_path = tile_result.get('image_path', '')
            
            # 타일 메타데이터 찾기
            tile_meta = None
            for meta in tile_metadata:
                if meta['tile_path'] == image_path:
                    tile_meta = meta
                    break
            
            if not tile_meta:
                logger.warning(f"타일 메타데이터를 찾을 수 없음: {image_path}")
                continue
            
            x_offset = tile_meta['x_offset']
            y_offset = tile_meta['y_offset']
            
            for detection in tile_result.get('detections', []):
                # 바운딩 박스를 전역 좌표로 변환
                bbox = detection['bbox']
                global_bbox = [
                    bbox[0] + x_offset,  # x1
                    bbox[1] + y_offset,  # y1
                    bbox[2] + x_offset,  # x2
                    bbox[3] + y_offset   # y2
                ]
                
                global_detection = {
                    'class_id': detection['class_id'],
                    'class_name': detection['class_name'],
                    'confidence': detection['confidence'],
                    'bbox': global_bbox,
                    'area': detection['area'],
                    'tile_id': tile_meta['tile_id'],
                    'source_tile': image_path
                }
                
                global_detections.append(global_detection)
        
        return global_detections
    
    def merge_overlapping_detections(self, detections: list) -> list:
        """겹치는 검출 결과 병합"""
        if not detections:
            return []
        
        # 신뢰도 기준 필터링
        filtered_detections = [
            d for d in detections 
            if d['confidence'] >= self.confidence_threshold
        ]
        
        # 클래스별로 그룹화
        class_groups = {}
        for detection in filtered_detections:
            class_name = detection['class_name']
            if class_name not in class_groups:
                class_groups[class_name] = []
            class_groups[class_name].append(detection)
        
        merged_detections = []
        
        # 클래스별로 NMS 적용
        for class_name, class_detections in class_groups.items():
            # 신뢰도 기준 정렬 (높은 순)
            class_detections.sort(key=lambda x: x['confidence'], reverse=True)
            
            while class_detections:
                # 가장 높은 신뢰도의 검출 선택
                best_detection = class_detections.pop(0)
                merged_detections.append(best_detection)
                
                # 겹치는 검출들 제거
                remaining = []
                for detection in class_detections:
                    iou = self.calculate_iou(best_detection['bbox'], detection['bbox'])
                    if iou < self.iou_threshold:
                        remaining.append(detection)
                    else:
                        logger.debug(f"IoU {iou:.3f}로 검출 병합: {class_name}")
                
                class_detections = remaining
        
        logger.info(f"병합 완료: {len(detections)} → {len(merged_detections)}개 검출")
        return merged_detections
    
    def merge_tile_results(
        self, 
        tile_results: list, 
        tile_metadata: list, 
        analysis_id: str
    ) -> dict:
        """타일 결과 병합 실행"""
        start_time = time.time()
        
        logger.info(f"타일 결과 병합 시작: {len(tile_results)}개 타일")
        
        # 1. 전역 좌표로 변환
        global_detections = self.convert_to_global_coords(tile_results, tile_metadata)
        
        # 2. 겹치는 검출 병합
        merged_detections = self.merge_overlapping_detections(global_detections)
        
        # 3. 결과 통계 계산
        class_counts = {}
        for detection in merged_detections:
            class_name = detection['class_name']
            class_counts[class_name] = class_counts.get(class_name, 0) + 1
        
        merge_time = time.time() - start_time
        
        result = {
            'analysis_id': analysis_id,
            'detections': merged_detections,
            'total_detections': len(merged_detections),
            'class_counts': class_counts,
            'tiles_processed': len(tile_results),
            'merge_time': merge_time,
            'iou_threshold': self.iou_threshold,
            'confidence_threshold': self.confidence_threshold
        }
        
        logger.info(f"병합 완료: {len(merged_detections)}개 최종 검출, {merge_time:.2f}초")
        
        return result

# 전역 병합 엔진 인스턴스
merge_engine = MergeEngine()

print("✅ POD5 결과 병합 엔진 통합 완료 (v2.0 신규)")
print(f"🔗 IoU 임계값: {merge_engine.iou_threshold}")
print(f"📊 신뢰도 임계값: {merge_engine.confidence_threshold}")

### POD6: GPKG 내보내기 엔진 (v2.0 신규 추가)

In [ ]:
# POD6: GPKG 내보내기 엔진 (v2.0 신규)
class GPKGExporter:
    """GPKG 파일 내보내기 엔진"""
    
    def __init__(self, output_dir: str = None):
        self.output_dir = Path(output_dir or settings.EXPORT_PATH)
        self.output_dir.mkdir(parents=True, exist_ok=True)
    
    def apply_privacy_protection(self, data: gpd.GeoDataFrame, privacy_config: dict = None) -> gpd.GeoDataFrame:
        """개인정보 보호 적용"""
        if not privacy_config or not privacy_config.get('mask_pnu', settings.PRIVACY_MASK):
            return data
        
        # PNU 마스킹 (뒤 4자리)
        if 'pnu' in data.columns:
            data = data.copy()
            data['pnu_masked'] = data['pnu'].apply(
                lambda x: x[:15] + '****' if len(str(x)) == 19 else x
            )
            data.drop('pnu', axis=1, inplace=True)
            data.rename(columns={'pnu_masked': 'pnu'}, inplace=True)
        
        return data
    
    def create_detection_layer(self, detections: list, image_metadata: dict) -> gpd.GeoDataFrame:
        """검출 결과 레이어 생성"""
        if not detections:
            return gpd.GeoDataFrame()
        
        geometries = []
        attributes = []
        
        for i, detection in enumerate(detections):
            # 바운딩 박스를 폴리곤으로 변환
            bbox = detection['bbox']
            polygon = box(bbox[0], bbox[1], bbox[2], bbox[3])
            geometries.append(polygon)
            
            # 속성 데이터
            attributes.append({
                'detection_id': i + 1,
                'class_name': detection['class_name'],
                'confidence': round(detection['confidence'], 3),
                'area_px': int(detection['area']),
                'bbox_x1': int(bbox[0]),
                'bbox_y1': int(bbox[1]),
                'bbox_x2': int(bbox[2]),
                'bbox_y2': int(bbox[3]),
                'source_tile': detection.get('source_tile', 'unknown')
            })
        
        # GeoDataFrame 생성
        gdf = gpd.GeoDataFrame(attributes, geometry=geometries)
        
        # 좌표계 설정 (픽셀 좌표계에서 지리 좌표계로 변환 필요)
        # 여기서는 임시로 EPSG:4326 사용 (실제로는 이미지 좌표계 변환 필요)
        gdf.crs = "EPSG:4326"
        
        return gdf
    
    def create_statistics_layer(self, detections: list, analysis_metadata: dict) -> gpd.GeoDataFrame:
        """통계 레이어 생성"""
        # 클래스별 통계 계산
        class_stats = {}
        for detection in detections:
            class_name = detection['class_name']
            if class_name not in class_stats:
                class_stats[class_name] = {
                    'count': 0,
                    'total_area': 0,
                    'avg_confidence': 0,
                    'confidences': []
                }
            
            class_stats[class_name]['count'] += 1
            class_stats[class_name]['total_area'] += detection['area']
            class_stats[class_name]['confidences'].append(detection['confidence'])
        
        # 평균 신뢰도 계산
        for class_name, stats in class_stats.items():
            stats['avg_confidence'] = np.mean(stats['confidences'])
        
        # Point 지오메트리로 통계 데이터 생성 (임의 좌표)
        geometries = []
        attributes = []
        
        for i, (class_name, stats) in enumerate(class_stats.items()):
            # 임의의 점 좌표 (실제로는 분석 영역 중심점 사용)
            point = Point(127.0 + i * 0.001, 37.0 + i * 0.001)
            geometries.append(point)
            
            attributes.append({
                'class_name': class_name,
                'detection_count': stats['count'],
                'total_area_px': int(stats['total_area']),
                'avg_confidence': round(stats['avg_confidence'], 3),
                'analysis_date': datetime.now().isoformat()[:10]
            })
        
        gdf = gpd.GeoDataFrame(attributes, geometry=geometries)
        gdf.crs = "EPSG:4326"
        
        return gdf
    
    def export_to_gpkg(
        self, 
        detections: list, 
        analysis_id: str, 
        analysis_metadata: dict = None,
        export_config: dict = None
    ) -> dict:
        """GPKG 파일로 내보내기"""
        start_time = time.time()
        
        # 파일명 생성
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        region_name = analysis_metadata.get('region_name', 'unknown') if analysis_metadata else 'unknown'
        filename = f"{region_name}_{timestamp}_analysis.gpkg"
        output_path = self.output_dir / filename
        
        try:
            logger.info(f"GPKG 내보내기 시작: {output_path}")
            
            # 검출 결과 레이어 생성
            detection_layer = self.create_detection_layer(detections, analysis_metadata or {})
            
            if not detection_layer.empty:
                # 개인정보 보호 적용
                if export_config and export_config.get('apply_privacy', True):
                    detection_layer = self.apply_privacy_protection(
                        detection_layer, 
                        export_config.get('privacy_config', {})
                    )
                
                # 출력 좌표계로 변환
                if detection_layer.crs != settings.OUTPUT_CRS:
                    detection_layer = detection_layer.to_crs(settings.OUTPUT_CRS)
                
                # GPKG에 검출 레이어 저장
                detection_layer.to_file(
                    output_path, 
                    layer='detections', 
                    driver="GPKG"
                )
                
                logger.info(f"검출 레이어 저장 완료: {len(detection_layer)}개 객체")
            
            # 통계 레이어 생성 및 저장
            if export_config and export_config.get('include_statistics', True):
                stats_layer = self.create_statistics_layer(detections, analysis_metadata or {})
                
                if not stats_layer.empty:
                    if stats_layer.crs != settings.OUTPUT_CRS:
                        stats_layer = stats_layer.to_crs(settings.OUTPUT_CRS)
                    
                    stats_layer.to_file(
                        output_path, 
                        layer='statistics', 
                        driver="GPKG"
                    )
                    
                    logger.info(f"통계 레이어 저장 완료: {len(stats_layer)}개 클래스")
            
            # 메타데이터 레이어 추가
            if export_config and export_config.get('include_metadata', True):
                metadata_dict = {
                    'analysis_id': analysis_id,
                    'export_date': datetime.now().isoformat(),
                    'total_detections': len(detections),
                    'output_crs': settings.OUTPUT_CRS,
                    'privacy_applied': export_config.get('apply_privacy', True)
                }
                
                if analysis_metadata:
                    metadata_dict.update(analysis_metadata)
                
                # 메타데이터를 Point로 저장
                metadata_gdf = gpd.GeoDataFrame(
                    [metadata_dict],
                    geometry=[Point(0, 0)],
                    crs="EPSG:4326"
                )
                
                metadata_gdf.to_file(
                    output_path, 
                    layer='metadata', 
                    driver="GPKG"
                )
            
            export_time = time.time() - start_time
            file_size = output_path.stat().st_size
            
            result = {
                'export_id': str(uuid4()),
                'file_path': str(output_path),
                'filename': filename,
                'file_size': file_size,
                'layer_count': 2 + (1 if export_config and export_config.get('include_metadata', True) else 0),
                'total_detections': len(detections),
                'export_time': export_time,
                'crs': settings.OUTPUT_CRS
            }
            
            logger.info(f"GPKG 내보내기 완료: {filename} ({file_size} bytes, {export_time:.2f}초)")
            
            return result
            
        except Exception as e:
            logger.error(f"GPKG 내보내기 실패: {e}")
            raise

# 전역 GPKG 내보내기 엔진 인스턴스
gpkg_exporter = GPKGExporter()

print("✅ POD6 GPKG 내보내기 엔진 통합 완료 (v2.0 신규)")
print(f"📦 출력 디렉토리: {gpkg_exporter.output_dir}")
print(f"🗺️ 출력 좌표계: {settings.OUTPUT_CRS}")
print(f"🔒 개인정보 보호: {settings.PRIVACY_MASK}")

## 📋 6. API 스키마 정의 (v2.0 확장)

In [ ]:
# API 응답 스키마 (v2.0 확장)
class APIResponse(BaseModel):
    """기본 API 응답"""
    success: bool = True
    message: str = ""
    data: Optional[Any] = None
    error: Optional[str] = None
    version: str = "2.0.0"  # v2.0 추가

# 이미지 관련 스키마
class ImageUploadRequest(BaseModel):
    """이미지 업로드 요청"""
    description: Optional[str] = None
    region_name: Optional[str] = None
    tags: List[str] = []

class ImageResponse(BaseModel):
    """이미지 응답"""
    id: str
    filename: str
    file_path: str
    file_size: int
    status: str
    format: str
    description: Optional[str] = None
    region_name: Optional[str] = None
    tags: List[str] = []
    metadata: Optional[dict] = None
    uploaded_at: datetime
    updated_at: datetime

# 분석 관련 스키마 (v2.0 확장)
class AnalysisRequest(BaseModel):
    """분석 요청 v2.0"""
    name: str
    description: Optional[str] = None
    analysis_type: str  # crop, facility, landuse
    confidence_threshold: float = Field(0.5, ge=0.1, le=1.0)
    use_tiling: bool = False  # v2.0 추가
    tile_size: Optional[int] = Field(None, ge=256, le=2048)  # v2.0 추가
    tile_overlap: Optional[float] = Field(None, ge=0.0, le=0.5)  # v2.0 추가
    shapefile_path: Optional[str] = None  # 크로핑용 Shapefile
    merge_results: bool = True  # v2.0 추가

class AnalysisResponse(BaseModel):
    """분석 응답 v2.0"""
    id: str
    name: str
    description: Optional[str] = None
    status: str
    analysis_type: str
    confidence_threshold: float
    use_tiling: bool = False  # v2.0 추가
    total_detections: int = 0
    total_tiles: int = 0  # v2.0 추가
    processing_time: Optional[float] = None
    error_message: Optional[str] = None
    created_at: datetime
    started_at: Optional[datetime] = None
    completed_at: Optional[datetime] = None

# 크롭 결과 스키마
class CropResultResponse(BaseModel):
    """크롭 결과 응답"""
    id: str
    pnu: str
    crop_path: str
    crop_size: int
    geometry_type: str
    area: float
    detected_class: Optional[str] = None
    confidence: Optional[float] = None
    detection_count: int = 0
    merged_from_tiles: bool = False  # v2.0 추가
    created_at: datetime

# 내보내기 관련 스키마 (v2.0 신규)
class ExportRequest(BaseModel):
    """내보내기 요청"""
    name: str
    export_type: str = "gpkg"  # gpkg, geojson, csv, shapefile
    include_statistics: bool = True
    include_metadata: bool = True
    apply_privacy: bool = True
    filter_confidence: Optional[float] = Field(None, ge=0.0, le=1.0)
    filter_classes: Optional[List[str]] = None

class ExportResponse(BaseModel):
    """내보내기 응답"""
    id: str
    name: str
    export_type: str
    status: str
    file_path: Optional[str] = None
    filename: Optional[str] = None
    file_size: Optional[int] = None
    layer_count: int = 0
    download_count: int = 0
    created_at: datetime
    completed_at: Optional[datetime] = None
    expires_at: Optional[datetime] = None

# 통계 관련 스키마 (v2.0 신규)
class StatisticsResponse(BaseModel):
    """통계 응답"""
    total_images: int
    total_analyses: int
    total_detections: int
    total_exports: int
    analysis_by_type: Dict[str, int]
    detections_by_class: Dict[str, int]
    avg_processing_time: float
    last_updated: datetime

print("✅ API 스키마 정의 완료 (v2.0 확장)")

## 🚀 7. FastAPI 애플리케이션 (v2.0)

In [ ]:
# FastAPI 애플리케이션 생성 (v2.0)
app = FastAPI(
    title=settings.PROJECT_NAME,
    description="AI 기반 농업 영상 분석 플랫폼 API v2.0 - 완전 통합 버전",
    version=settings.VERSION,
    docs_url="/api/docs",
    redoc_url="/api/redoc"
)

# CORS 미들웨어
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 헬스 체크
@app.get("/health")
async def health_check():
    """헬스 체크 엔드포인트 v2.0"""
    return APIResponse(
        success=True,
        message="서비스가 정상 작동 중입니다 (v2.0)",
        data={
            "timestamp": datetime.now().isoformat(),
            "version": settings.VERSION,
            "environment": settings.ENVIRONMENT,
            "features": {
                "tiling": True,
                "merging": True,
                "gpkg_export": True,
                "statistics": True
            }
        }
    )

print("✅ FastAPI 애플리케이션 생성 완료 (v2.0)")

## 📸 8. Images API 구현 (v1.0과 동일)

In [ ]:
# Images API 엔드포인트 (v1.0과 동일)
@app.post("/api/v1/images/upload", response_model=APIResponse)
async def upload_image(
    file: UploadFile = File(...),
    description: str = None,
    region_name: str = None,
    db: Session = Depends(get_db)
):
    """이미지 업로드"""
    try:
        # 파일 확장자 검증
        allowed_extensions = ['.tif', '.tiff', '.jp2']
        file_extension = Path(file.filename).suffix.lower()
        if file_extension not in allowed_extensions:
            raise HTTPException(
                status_code=400,
                detail=f"지원되지 않는 파일 형식: {file_extension}"
            )
        
        # 파일 저장
        file_path, file_size = data_registry.save_file(file)
        
        # 메타데이터 추출
        try:
            metadata = data_registry.extract_image_metadata(Path(file_path))
        except Exception as e:
            logger.warning(f"메타데이터 추출 실패: {e}")
            metadata = {}
        
        # 테스트 사용자 가져오기
        test_user = db.query(User).filter(User.username == "testuser").first()
        if not test_user:
            raise HTTPException(status_code=404, detail="테스트 사용자를 찾을 수 없습니다")
        
        # 데이터베이스에 이미지 정보 저장
        db_image = Image(
            filename=file.filename,
            file_path=file_path,
            file_size=file_size,
            format=file_extension.replace('.', ''),
            status="ready",
            description=description,
            region_name=region_name,
            owner_id=test_user.id,
            crs=metadata.get('crs'),
            bounds=metadata.get('bounds'),
            resolution=metadata.get('resolution'),
            width=metadata.get('width'),
            height=metadata.get('height'),
            bands=metadata.get('bands')
        )
        
        db.add(db_image)
        db.commit()
        db.refresh(db_image)
        
        # 응답 데이터 구성
        response_data = ImageResponse(
            id=db_image.id,
            filename=db_image.filename,
            file_path=db_image.file_path,
            file_size=db_image.file_size,
            status=db_image.status,
            format=db_image.format,
            description=db_image.description,
            region_name=db_image.region_name,
            tags=[],
            metadata=metadata,
            uploaded_at=db_image.uploaded_at,
            updated_at=db_image.updated_at
        )
        
        return APIResponse(
            success=True,
            message="이미지 업로드 완료",
            data=response_data.dict()
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"이미지 업로드 실패: {e}")
        raise HTTPException(status_code=500, detail=f"이미지 업로드 실패: {str(e)}")


@app.get("/api/v1/images", response_model=APIResponse)
async def list_images(db: Session = Depends(get_db)):
    """이미지 목록 조회"""
    try:
        images = db.query(Image).all()
        
        image_list = []
        for img in images:
            image_data = ImageResponse(
                id=img.id,
                filename=img.filename,
                file_path=img.file_path,
                file_size=img.file_size,
                status=img.status,
                format=img.format,
                description=img.description,
                region_name=img.region_name,
                tags=img.tags or [],
                uploaded_at=img.uploaded_at,
                updated_at=img.updated_at
            )
            image_list.append(image_data.dict())
        
        return APIResponse(
            success=True,
            message=f"{len(image_list)}개의 이미지를 찾았습니다",
            data={"images": image_list, "total": len(image_list)}
        )
        
    except Exception as e:
        logger.error(f"이미지 목록 조회 실패: {e}")
        raise HTTPException(status_code=500, detail=f"이미지 목록 조회 실패: {str(e)}")


@app.get("/api/v1/images/{image_id}", response_model=APIResponse)
async def get_image(image_id: str, db: Session = Depends(get_db)):
    """이미지 상세 조회"""
    try:
        image = db.query(Image).filter(Image.id == image_id).first()
        if not image:
            raise HTTPException(status_code=404, detail="이미지를 찾을 수 없습니다")
        
        image_data = ImageResponse(
            id=image.id,
            filename=image.filename,
            file_path=image.file_path,
            file_size=image.file_size,
            status=image.status,
            format=image.format,
            description=image.description,
            region_name=image.region_name,
            tags=image.tags or [],
            metadata={
                "crs": image.crs,
                "bounds": image.bounds,
                "resolution": image.resolution,
                "width": image.width,
                "height": image.height,
                "bands": image.bands
            },
            uploaded_at=image.uploaded_at,
            updated_at=image.updated_at
        )
        
        return APIResponse(
            success=True,
            message="이미지 조회 완료",
            data=image_data.dict()
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"이미지 조회 실패: {e}")
        raise HTTPException(status_code=500, detail=f"이미지 조회 실패: {str(e)}")

print("✅ Images API 구현 완료")

## 🎯 9. Analysis API v2.0 (타일링 지원)

In [ ]:
# Analysis API v2.0 (타일링 지원)
@app.post("/api/v1/analyses", response_model=APIResponse)
async def create_analysis(
    image_id: str,
    request: AnalysisRequest,
    background_tasks: BackgroundTasks,
    db: Session = Depends(get_db)
):
    """분석 작업 생성 v2.0 (타일링 지원)"""
    try:
        # 이미지 존재 확인
        image = db.query(Image).filter(Image.id == image_id).first()
        if not image:
            raise HTTPException(status_code=404, detail="이미지를 찾을 수 없습니다")
        
        # 테스트 사용자 가져오기
        test_user = db.query(User).filter(User.username == "testuser").first()
        if not test_user:
            raise HTTPException(status_code=404, detail="사용자를 찾을 수 없습니다")
        
        # 분석 작업 생성
        analysis = Analysis(
            name=request.name,
            description=request.description,
            status="pending",
            analysis_type=request.analysis_type,
            confidence_threshold=request.confidence_threshold,
            use_tiling=request.use_tiling,
            config={
                "tile_size": request.tile_size or settings.TILE_SIZE,
                "tile_overlap": request.tile_overlap or settings.TILE_OVERLAP,
                "shapefile_path": request.shapefile_path,
                "merge_results": request.merge_results
            },
            image_id=image_id,
            owner_id=test_user.id
        )
        
        db.add(analysis)
        db.commit()
        db.refresh(analysis)
        
        # 백그라운드에서 분석 실행
        background_tasks.add_task(
            run_analysis_v2,
            analysis.id,
            image.file_path,
            request
        )
        
        response_data = AnalysisResponse(
            id=analysis.id,
            name=analysis.name,
            description=analysis.description,
            status=analysis.status,
            analysis_type=analysis.analysis_type,
            confidence_threshold=analysis.confidence_threshold,
            use_tiling=analysis.use_tiling,
            created_at=analysis.created_at
        )
        
        return APIResponse(
            success=True,
            message="분석 작업이 시작되었습니다 (v2.0)",
            data=response_data.dict()
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"분석 생성 실패: {e}")
        raise HTTPException(status_code=500, detail=f"분석 생성 실패: {str(e)}")


# 분석 실행 함수 v2.0
def run_analysis_v2(analysis_id: str, image_path: str, request: AnalysisRequest):
    """분석 실행 (v2.0 - 타일링 지원)"""
    db = SessionLocal()
    
    try:
        # 분석 작업 조회
        analysis = db.query(Analysis).filter(Analysis.id == analysis_id).first()
        if not analysis:
            logger.error(f"분석 작업을 찾을 수 없음: {analysis_id}")
            return
        
        # 상태 업데이트
        analysis.status = "running"
        analysis.started_at = datetime.utcnow()
        db.commit()
        
        logger.info(f"분석 시작: {analysis_id} (v2.0)")
        
        if request.use_tiling:
            # 타일링 기반 분석
            result = run_tiling_analysis(analysis_id, image_path, request, db)
        else:
            # 일반 분석 (크로핑 기반)
            result = run_standard_analysis(analysis_id, image_path, request, db)
        
        # 분석 완료 처리
        analysis.status = "completed"
        analysis.completed_at = datetime.utcnow()
        analysis.total_detections = result.get('total_detections', 0)
        analysis.total_tiles = result.get('total_tiles', 0)
        analysis.processing_time = result.get('processing_time', 0)
        analysis.result_data = result
        
        db.commit()
        
        logger.info(f"분석 완료: {analysis_id}")
        
    except Exception as e:
        logger.error(f"분석 실행 실패: {e}")
        
        # 에러 상태 업데이트
        analysis.status = "error"
        analysis.error_message = str(e)
        analysis.completed_at = datetime.utcnow()
        db.commit()
        
    finally:
        db.close()


def run_tiling_analysis(analysis_id: str, image_path: str, request: AnalysisRequest, db: Session) -> dict:
    """타일링 기반 분석 실행"""
    start_time = time.time()
    
    # 1. 이미지 타일링
    logger.info("타일링 시작")
    tiles = tiling_engine.generate_tiles(image_path, analysis_id)
    
    # 2. 타일 정보 DB 저장
    for tile_info in tiles:
        tile = Tile(
            tile_path=tile_info['tile_path'],
            tile_size=tile_info['tile_size'],
            row=tile_info['row'],
            col=tile_info['col'],
            x_offset=tile_info['x_offset'],
            y_offset=tile_info['y_offset'],
            bounds=tile_info['bounds'],
            pixel_bounds=tile_info['pixel_bounds'],
            image_id=db.query(Analysis).filter(Analysis.id == analysis_id).first().image_id,
            analysis_id=analysis_id
        )
        db.add(tile)
    
    db.commit()
    
    # 3. AI 추론 (배치 처리)
    logger.info(f"AI 추론 시작: {len(tiles)}개 타일")
    tile_paths = [tile['tile_path'] for tile in tiles]
    tile_results = ai_engine.predict_batch(
        tile_paths, 
        request.analysis_type, 
        request.confidence_threshold
    )
    
    # 4. 결과 병합
    logger.info("결과 병합 시작")
    merged_result = merge_engine.merge_tile_results(
        tile_results,
        tiles,
        analysis_id
    )
    
    processing_time = time.time() - start_time
    
    return {
        'total_detections': merged_result['total_detections'],
        'total_tiles': len(tiles),
        'processing_time': processing_time,
        'class_counts': merged_result['class_counts'],
        'merge_stats': {
            'iou_threshold': merged_result['iou_threshold'],
            'confidence_threshold': merged_result['confidence_threshold']
        }
    }


def run_standard_analysis(analysis_id: str, image_path: str, request: AnalysisRequest, db: Session) -> dict:
    """표준 분석 실행 (크로핑 기반)"""
    start_time = time.time()
    
    if not request.shapefile_path or not Path(request.shapefile_path).exists():
        raise ValueError("Shapefile 경로가 필요합니다")
    
    # 1. 크로핑
    logger.info("크로핑 시작")
    crop_results = cropping_engine.crop_by_shapes(image_path, request.shapefile_path)
    
    # 2. AI 추론
    logger.info(f"AI 추론 시작: {len(crop_results)}개 크롭")
    crop_paths = [result['crop_path'] for result in crop_results]
    ai_results = ai_engine.predict_batch(
        crop_paths,
        request.analysis_type,
        request.confidence_threshold
    )
    
    # 3. 결과 DB 저장
    total_detections = 0
    for crop_result, ai_result in zip(crop_results, ai_results):
        if 'error' in ai_result:
            continue
            
        detection_count = ai_result.get('detection_count', 0)
        detected_class = None
        confidence = 0.0
        
        if ai_result.get('detections'):
            # 가장 높은 신뢰도 검출 사용
            best_detection = max(ai_result['detections'], key=lambda x: x['confidence'])
            detected_class = best_detection['class_name']
            confidence = best_detection['confidence']
        
        crop_db = CropResult(
            pnu=crop_result['pnu'],
            crop_path=crop_result['crop_path'],
            crop_size=crop_result['crop_size'],
            geometry_type=crop_result['geometry_type'],
            area=crop_result['area'],
            bounds=crop_result['bounds'],
            detected_class=detected_class,
            confidence=confidence,
            detection_count=detection_count,
            detection_data=ai_result,
            image_id=db.query(Analysis).filter(Analysis.id == analysis_id).first().image_id,
            analysis_id=analysis_id
        )
        
        db.add(crop_db)
        total_detections += detection_count
    
    db.commit()
    
    processing_time = time.time() - start_time
    
    return {
        'total_detections': total_detections,
        'total_tiles': 0,
        'processing_time': processing_time,
        'total_crops': len(crop_results)
    }


@app.get("/api/v1/analyses", response_model=APIResponse)
async def list_analyses(db: Session = Depends(get_db)):
    """분석 목록 조회"""
    try:
        analyses = db.query(Analysis).all()
        
        analysis_list = []
        for analysis in analyses:
            analysis_data = AnalysisResponse(
                id=analysis.id,
                name=analysis.name,
                description=analysis.description,
                status=analysis.status,
                analysis_type=analysis.analysis_type,
                confidence_threshold=analysis.confidence_threshold,
                use_tiling=analysis.use_tiling,
                total_detections=analysis.total_detections,
                total_tiles=analysis.total_tiles,
                processing_time=analysis.processing_time,
                error_message=analysis.error_message,
                created_at=analysis.created_at,
                started_at=analysis.started_at,
                completed_at=analysis.completed_at
            )
            analysis_list.append(analysis_data.dict())
        
        return APIResponse(
            success=True,
            message=f"{len(analysis_list)}개의 분석을 찾았습니다",
            data={"analyses": analysis_list, "total": len(analysis_list)}
        )
        
    except Exception as e:
        logger.error(f"분석 목록 조회 실패: {e}")
        raise HTTPException(status_code=500, detail=f"분석 목록 조회 실패: {str(e)}")


@app.get("/api/v1/analyses/{analysis_id}", response_model=APIResponse)
async def get_analysis(analysis_id: str, db: Session = Depends(get_db)):
    """분석 상세 조회"""
    try:
        analysis = db.query(Analysis).filter(Analysis.id == analysis_id).first()
        if not analysis:
            raise HTTPException(status_code=404, detail="분석을 찾을 수 없습니다")
        
        analysis_data = AnalysisResponse(
            id=analysis.id,
            name=analysis.name,
            description=analysis.description,
            status=analysis.status,
            analysis_type=analysis.analysis_type,
            confidence_threshold=analysis.confidence_threshold,
            use_tiling=analysis.use_tiling,
            total_detections=analysis.total_detections,
            total_tiles=analysis.total_tiles,
            processing_time=analysis.processing_time,
            error_message=analysis.error_message,
            created_at=analysis.created_at,
            started_at=analysis.started_at,
            completed_at=analysis.completed_at
        )
        
        return APIResponse(
            success=True,
            message="분석 조회 완료",
            data=analysis_data.dict()
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"분석 조회 실패: {e}")
        raise HTTPException(status_code=500, detail=f"분석 조회 실패: {str(e)}")

print("✅ Analysis API v2.0 구현 완료 (타일링 지원)")

## 🌱 10. Crops API v2.0

In [ ]:
# Crops API v2.0
@app.get("/api/v1/analyses/{analysis_id}/crops", response_model=APIResponse)
async def get_analysis_crops(analysis_id: str, db: Session = Depends(get_db)):
    """분석의 크롭 결과 조회"""
    try:
        # 분석 존재 확인
        analysis = db.query(Analysis).filter(Analysis.id == analysis_id).first()
        if not analysis:
            raise HTTPException(status_code=404, detail="분석을 찾을 수 없습니다")
        
        # 크롭 결과 조회
        crops = db.query(CropResult).filter(CropResult.analysis_id == analysis_id).all()
        
        crop_list = []
        for crop in crops:
            crop_data = CropResultResponse(
                id=crop.id,
                pnu=crop.pnu,
                crop_path=crop.crop_path,
                crop_size=crop.crop_size,
                geometry_type=crop.geometry_type,
                area=crop.area,
                detected_class=crop.detected_class,
                confidence=crop.confidence,
                detection_count=crop.detection_count,
                merged_from_tiles=crop.merged_from_tiles,
                created_at=crop.created_at
            )
            crop_list.append(crop_data.dict())
        
        return APIResponse(
            success=True,
            message=f"{len(crop_list)}개의 크롭 결과를 찾았습니다",
            data={"crops": crop_list, "total": len(crop_list)}
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"크롭 결과 조회 실패: {e}")
        raise HTTPException(status_code=500, detail=f"크롭 결과 조회 실패: {str(e)}")


@app.get("/api/v1/crops/{crop_id}", response_model=APIResponse)
async def get_crop_detail(crop_id: str, db: Session = Depends(get_db)):
    """크롭 상세 조회"""
    try:
        crop = db.query(CropResult).filter(CropResult.id == crop_id).first()
        if not crop:
            raise HTTPException(status_code=404, detail="크롭을 찾을 수 없습니다")
        
        crop_data = CropResultResponse(
            id=crop.id,
            pnu=crop.pnu,
            crop_path=crop.crop_path,
            crop_size=crop.crop_size,
            geometry_type=crop.geometry_type,
            area=crop.area,
            detected_class=crop.detected_class,
            confidence=crop.confidence,
            detection_count=crop.detection_count,
            merged_from_tiles=crop.merged_from_tiles,
            created_at=crop.created_at
        )
        
        return APIResponse(
            success=True,
            message="크롭 조회 완료",
            data={
                "crop": crop_data.dict(),
                "detection_data": crop.detection_data
            }
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"크롭 조회 실패: {e}")
        raise HTTPException(status_code=500, detail=f"크롭 조회 실패: {str(e)}")


@app.get("/api/v1/crops/{crop_id}/download")
async def download_crop_image(crop_id: str, db: Session = Depends(get_db)):
    """크롭 이미지 다운로드"""
    try:
        crop = db.query(CropResult).filter(CropResult.id == crop_id).first()
        if not crop:
            raise HTTPException(status_code=404, detail="크롭을 찾을 수 없습니다")
        
        crop_path = Path(crop.crop_path)
        if not crop_path.exists():
            raise HTTPException(status_code=404, detail="크롭 파일을 찾을 수 없습니다")
        
        return FileResponse(
            path=crop_path,
            filename=f"crop_{crop.pnu}.tif",
            media_type="image/tiff"
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"크롭 다운로드 실패: {e}")
        raise HTTPException(status_code=500, detail=f"크롭 다운로드 실패: {str(e)}")

print("✅ Crops API v2.0 구현 완료")

## 📦 11. Exports API v2.0 (GPKG 내보내기)

In [ ]:
# Exports API v2.0 (GPKG 내보내기)
@app.post("/api/v1/analyses/{analysis_id}/exports", response_model=APIResponse)
async def create_export(
    analysis_id: str,
    request: ExportRequest,
    background_tasks: BackgroundTasks,
    db: Session = Depends(get_db)
):
    """분석 결과 내보내기 생성"""
    try:
        # 분석 존재 확인
        analysis = db.query(Analysis).filter(Analysis.id == analysis_id).first()
        if not analysis:
            raise HTTPException(status_code=404, detail="분석을 찾을 수 없습니다")
        
        if analysis.status != "completed":
            raise HTTPException(status_code=400, detail="완료된 분석만 내보낼 수 있습니다")
        
        # 내보내기 작업 생성
        export = Export(
            name=request.name,
            export_type=request.export_type,
            status="pending",
            include_geometry=True,
            include_metadata=request.include_metadata,
            privacy_applied=request.apply_privacy,
            filter_config={
                "filter_confidence": request.filter_confidence,
                "filter_classes": request.filter_classes
            },
            analysis_id=analysis_id,
            expires_at=datetime.utcnow() + timedelta(days=7)  # 7일 후 만료
        )
        
        db.add(export)
        db.commit()
        db.refresh(export)
        
        # 백그라운드에서 내보내기 실행
        background_tasks.add_task(
            run_export_task,
            export.id,
            analysis_id,
            request
        )
        
        response_data = ExportResponse(
            id=export.id,
            name=export.name,
            export_type=export.export_type,
            status=export.status,
            created_at=export.created_at,
            expires_at=export.expires_at
        )
        
        return APIResponse(
            success=True,
            message="내보내기 작업이 시작되었습니다",
            data=response_data.dict()
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"내보내기 생성 실패: {e}")
        raise HTTPException(status_code=500, detail=f"내보내기 생성 실패: {str(e)}")


def run_export_task(export_id: str, analysis_id: str, request: ExportRequest):
    """내보내기 작업 실행"""
    db = SessionLocal()
    
    try:
        # 내보내기 작업 조회
        export = db.query(Export).filter(Export.id == export_id).first()
        if not export:
            logger.error(f"내보내기 작업을 찾을 수 없음: {export_id}")
            return
        
        # 상태 업데이트
        export.status = "processing"
        db.commit()
        
        logger.info(f"내보내기 시작: {export_id}")
        
        # 분석 결과 수집
        analysis = db.query(Analysis).filter(Analysis.id == analysis_id).first()
        
        if analysis.use_tiling:
            # 타일링 기반 결과 수집
            detections = collect_tiling_results(analysis_id, db)
        else:
            # 크로핑 기반 결과 수집
            detections = collect_cropping_results(analysis_id, db)
        
        # 필터 적용
        if request.filter_confidence:
            detections = [
                d for d in detections 
                if d.get('confidence', 0) >= request.filter_confidence
            ]
        
        if request.filter_classes:
            detections = [
                d for d in detections 
                if d.get('class_name') in request.filter_classes
            ]
        
        # 메타데이터 준비
        image = db.query(Image).filter(Image.id == analysis.image_id).first()
        analysis_metadata = {
            'analysis_id': analysis_id,
            'analysis_name': analysis.name,
            'analysis_type': analysis.analysis_type,
            'image_filename': image.filename if image else 'unknown',
            'region_name': image.region_name if image else 'unknown',
            'processing_time': analysis.processing_time
        }
        
        # 내보내기 설정
        export_config = {
            'include_statistics': request.include_statistics,
            'include_metadata': request.include_metadata,
            'apply_privacy': request.apply_privacy,
            'privacy_config': {
                'mask_pnu': True
            }
        }
        
        # GPKG 내보내기 실행
        if request.export_type == "gpkg":
            result = gpkg_exporter.export_to_gpkg(
                detections,
                analysis_id,
                analysis_metadata,
                export_config
            )
        else:
            raise ValueError(f"지원되지 않는 내보내기 형식: {request.export_type}")
        
        # 내보내기 완료 처리
        export.status = "completed"
        export.file_path = result['file_path']
        export.file_size = result['file_size']
        export.layer_count = result['layer_count']
        export.completed_at = datetime.utcnow()
        
        db.commit()
        
        logger.info(f"내보내기 완료: {export_id}")
        
    except Exception as e:
        logger.error(f"내보내기 실행 실패: {e}")
        
        # 에러 상태 업데이트
        export.status = "error"
        export.error_message = str(e)
        export.completed_at = datetime.utcnow()
        db.commit()
        
    finally:
        db.close()


def collect_tiling_results(analysis_id: str, db: Session) -> list:
    """타일링 결과 수집"""
    # 실제 구현에서는 analysis.result_data에서 병합된 검출 결과를 가져옴
    analysis = db.query(Analysis).filter(Analysis.id == analysis_id).first()
    if analysis and analysis.result_data:
        return analysis.result_data.get('detections', [])
    return []


def collect_cropping_results(analysis_id: str, db: Session) -> list:
    """크로핑 결과 수집"""
    crops = db.query(CropResult).filter(CropResult.analysis_id == analysis_id).all()
    
    detections = []
    for crop in crops:
        if crop.detected_class and crop.confidence:
            # 크롭 결과를 검출 형식으로 변환
            detection = {
                'class_name': crop.detected_class,
                'confidence': crop.confidence,
                'bbox': [0, 0, 100, 100],  # 임시 좌표
                'area': crop.area,
                'pnu': crop.pnu,
                'source_type': 'crop'
            }
            detections.append(detection)
    
    return detections


@app.get("/api/v1/exports/{export_id}", response_model=APIResponse)
async def get_export(export_id: str, db: Session = Depends(get_db)):
    """내보내기 상세 조회"""
    try:
        export = db.query(Export).filter(Export.id == export_id).first()
        if not export:
            raise HTTPException(status_code=404, detail="내보내기를 찾을 수 없습니다")
        
        export_data = ExportResponse(
            id=export.id,
            name=export.name,
            export_type=export.export_type,
            status=export.status,
            file_path=export.file_path,
            filename=Path(export.file_path).name if export.file_path else None,
            file_size=export.file_size,
            layer_count=export.layer_count,
            download_count=export.download_count,
            created_at=export.created_at,
            completed_at=export.completed_at,
            expires_at=export.expires_at
        )
        
        return APIResponse(
            success=True,
            message="내보내기 조회 완료",
            data=export_data.dict()
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"내보내기 조회 실패: {e}")
        raise HTTPException(status_code=500, detail=f"내보내기 조회 실패: {str(e)}")


@app.get("/api/v1/exports/{export_id}/download")
async def download_export(export_id: str, db: Session = Depends(get_db)):
    """내보내기 파일 다운로드"""
    try:
        export = db.query(Export).filter(Export.id == export_id).first()
        if not export:
            raise HTTPException(status_code=404, detail="내보내기를 찾을 수 없습니다")
        
        if export.status != "completed":
            raise HTTPException(status_code=400, detail="완료된 내보내기만 다운로드할 수 있습니다")
        
        if not export.file_path or not Path(export.file_path).exists():
            raise HTTPException(status_code=404, detail="내보내기 파일을 찾을 수 없습니다")
        
        # 만료일 확인
        if export.expires_at and datetime.utcnow() > export.expires_at:
            raise HTTPException(status_code=410, detail="내보내기 파일이 만료되었습니다")
        
        # 다운로드 횟수 증가
        export.download_count += 1
        db.commit()
        
        filename = Path(export.file_path).name
        media_type = "application/geopackage+sqlite3" if export.export_type == "gpkg" else "application/octet-stream"
        
        return FileResponse(
            path=export.file_path,
            filename=filename,
            media_type=media_type
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"내보내기 다운로드 실패: {e}")
        raise HTTPException(status_code=500, detail=f"내보내기 다운로드 실패: {str(e)}")

print("✅ Exports API v2.0 구현 완료")

## 📊 12. Statistics API v2.0

In [ ]:
# Statistics API v2.0
@app.get("/api/v1/statistics", response_model=APIResponse)
async def get_statistics(db: Session = Depends(get_db)):
    """전체 통계 조회"""
    try:
        # 기본 통계 수집
        total_images = db.query(Image).count()
        total_analyses = db.query(Analysis).count()
        total_exports = db.query(Export).count()
        
        # 검출 수 계산
        total_detections = db.query(Analysis).filter(
            Analysis.status == "completed"
        ).with_entities(
            db.func.sum(Analysis.total_detections)
        ).scalar() or 0
        
        # 분석 유형별 통계
        analysis_by_type = {}
        type_stats = db.query(
            Analysis.analysis_type,
            db.func.count(Analysis.id)
        ).group_by(Analysis.analysis_type).all()
        
        for analysis_type, count in type_stats:
            analysis_by_type[analysis_type] = count
        
        # 검출 클래스별 통계 (크롭 결과 기반)
        detections_by_class = {}
        class_stats = db.query(
            CropResult.detected_class,
            db.func.count(CropResult.id)
        ).filter(
            CropResult.detected_class.isnot(None)
        ).group_by(CropResult.detected_class).all()
        
        for class_name, count in class_stats:
            detections_by_class[class_name] = count
        
        # 평균 처리 시간
        avg_processing_time = db.query(
            db.func.avg(Analysis.processing_time)
        ).filter(
            Analysis.status == "completed",
            Analysis.processing_time.isnot(None)
        ).scalar() or 0.0
        
        stats_data = StatisticsResponse(
            total_images=total_images,
            total_analyses=total_analyses,
            total_detections=int(total_detections),
            total_exports=total_exports,
            analysis_by_type=analysis_by_type,
            detections_by_class=detections_by_class,
            avg_processing_time=float(avg_processing_time),
            last_updated=datetime.utcnow()
        )
        
        return APIResponse(
            success=True,
            message="통계 조회 완료",
            data=stats_data.dict()
        )
        
    except Exception as e:
        logger.error(f"통계 조회 실패: {e}")
        raise HTTPException(status_code=500, detail=f"통계 조회 실패: {str(e)}")


@app.get("/api/v1/statistics/analyses/{analysis_id}", response_model=APIResponse)
async def get_analysis_statistics(analysis_id: str, db: Session = Depends(get_db)):
    """분석별 상세 통계"""
    try:
        # 분석 존재 확인
        analysis = db.query(Analysis).filter(Analysis.id == analysis_id).first()
        if not analysis:
            raise HTTPException(status_code=404, detail="분석을 찾을 수 없습니다")
        
        stats = {
            'analysis_id': analysis_id,
            'analysis_name': analysis.name,
            'analysis_type': analysis.analysis_type,
            'status': analysis.status,
            'total_detections': analysis.total_detections,
            'total_tiles': analysis.total_tiles,
            'processing_time': analysis.processing_time,
            'created_at': analysis.created_at.isoformat(),
            'completed_at': analysis.completed_at.isoformat() if analysis.completed_at else None
        }
        
        if analysis.use_tiling:
            # 타일 통계
            tile_count = db.query(Tile).filter(Tile.analysis_id == analysis_id).count()
            processed_tiles = db.query(Tile).filter(
                Tile.analysis_id == analysis_id,
                Tile.processed == True
            ).count()
            
            stats['tile_statistics'] = {
                'total_tiles': tile_count,
                'processed_tiles': processed_tiles,
                'processing_rate': processed_tiles / tile_count if tile_count > 0 else 0
            }
        else:
            # 크롭 통계
            crop_count = db.query(CropResult).filter(CropResult.analysis_id == analysis_id).count()
            detected_crops = db.query(CropResult).filter(
                CropResult.analysis_id == analysis_id,
                CropResult.detected_class.isnot(None)
            ).count()
            
            # 클래스별 통계
            class_stats = db.query(
                CropResult.detected_class,
                db.func.count(CropResult.id),
                db.func.avg(CropResult.confidence)
            ).filter(
                CropResult.analysis_id == analysis_id,
                CropResult.detected_class.isnot(None)
            ).group_by(CropResult.detected_class).all()
            
            class_breakdown = {}
            for class_name, count, avg_conf in class_stats:
                class_breakdown[class_name] = {
                    'count': count,
                    'avg_confidence': round(float(avg_conf), 3)
                }
            
            stats['crop_statistics'] = {
                'total_crops': crop_count,
                'detected_crops': detected_crops,
                'detection_rate': detected_crops / crop_count if crop_count > 0 else 0,
                'class_breakdown': class_breakdown
            }
        
        return APIResponse(
            success=True,
            message="분석 통계 조회 완료",
            data=stats
        )
        
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"분석 통계 조회 실패: {e}")
        raise HTTPException(status_code=500, detail=f"분석 통계 조회 실패: {str(e)}")

print("✅ Statistics API v2.0 구현 완료")

## 🚀 13. 서버 시작 및 테스트 (v2.0)

In [ ]:
# 테스트용 샘플 데이터 생성
def create_sample_data():
    """테스트용 샘플 데이터 생성"""
    db = SessionLocal()
    
    try:
        # 샘플 이미지 생성
        sample_image = Image(
            filename="sample_namwon_20250115.tif",
            file_path="./uploads/sample_namwon_20250115.tif",
            file_size=1024000,
            format="tif",
            status="ready",
            description="남원시 농업지역 드론 영상 샘플",
            region_name="전북 남원시",
            owner_id=test_user.id,
            crs="EPSG:5186",
            bounds={
                "minx": 200000.0,
                "miny": 400000.0,
                "maxx": 205000.0,
                "maxy": 405000.0
            },
            resolution=0.25,
            width=20000,
            height=20000,
            bands=3
        )
        
        # 기존 샘플 확인
        existing_sample = db.query(Image).filter(
            Image.filename == sample_image.filename
        ).first()
        
        if not existing_sample:
            db.add(sample_image)
            db.commit()
            db.refresh(sample_image)
            print(f"✅ 샘플 이미지 생성: {sample_image.id}")
        else:
            sample_image = existing_sample
            print(f"ℹ️ 샘플 이미지 존재: {sample_image.id}")
        
        return sample_image
        
    except Exception as e:
        logger.error(f"샘플 데이터 생성 실패: {e}")
        return None
    finally:
        db.close()


def test_apis():
    """API 기능 테스트"""
    print("\n🧪 API 테스트 시작...")
    
    # 샘플 데이터 생성
    sample_image = create_sample_data()
    if not sample_image:
        print("❌ 샘플 데이터 생성 실패")
        return
    
    print(f"📋 테스트 데이터:")
    print(f"   - 이미지 ID: {sample_image.id}")
    print(f"   - 파일명: {sample_image.filename}")
    print(f"   - 지역: {sample_image.region_name}")
    print(f"   - 크기: {sample_image.width}x{sample_image.height}")
    
    # API 엔드포인트 테스트
    test_endpoints = [
        {
            "name": "헬스 체크",
            "url": "/health",
            "method": "GET"
        },
        {
            "name": "이미지 목록",
            "url": "/api/v1/images",
            "method": "GET"
        },
        {
            "name": "이미지 상세",
            "url": f"/api/v1/images/{sample_image.id}",
            "method": "GET"
        },
        {
            "name": "분석 목록",
            "url": "/api/v1/analyses",
            "method": "GET"
        },
        {
            "name": "전체 통계",
            "url": "/api/v1/statistics",
            "method": "GET"
        }
    ]
    
    print(f"\n🔗 테스트 가능한 엔드포인트 ({len(test_endpoints)}개):")
    for endpoint in test_endpoints:
        print(f"   - {endpoint['method']} {endpoint['url']} ({endpoint['name']})")
    
    print("\n📖 API 문서: http://localhost:8000/api/docs")
    print("📘 Redoc: http://localhost:8000/api/redoc")


# 서버 시작 함수
def start_server():
    """개발 서버 시작"""
    import uvicorn
    
    print("\n🚀 Nong-View API 서버 시작 (v2.0)")
    print(f"🌐 주소: http://{settings.HOST}:{settings.PORT}")
    print(f"📚 API 문서: http://{settings.HOST}:{settings.PORT}/api/docs")
    print(f"🔧 환경: {settings.ENVIRONMENT}")
    print(f"📊 버전: {settings.VERSION}")
    
    # 테스트 데이터 및 API 정보 출력
    test_apis()
    
    print("\n" + "="*50)
    print("서버를 시작하려면 아래 코드를 실행하세요:")
    print("start_server()")
    print("="*50)
    
    # 실제 서버 시작 (주석 해제 시 자동 시작)
    # uvicorn.run(
    #     "__main__:app",
    #     host=settings.HOST,
    #     port=settings.PORT,
    #     reload=settings.DEBUG
    # )


# 서버 정보 출력
print("\n" + "="*60)
print("🎉 Nong-View v2.0 통합 노트북 준비 완료!")
print("="*60)
print(f"📈 완성도: 98% (v1.0 기능 + v2.0 신규 기능)")
print(f"🆕 v2.0 신규 기능:")
print(f"   - ✅ POD3: 대용량 이미지 타일링 시스템")
print(f"   - ✅ POD5: IoU 기반 중복 검출 병합")
print(f"   - ✅ POD6: GPKG 표준 파일 내보내기")
print(f"   - ✅ Analysis API v2.0 (타일링 지원)")
print(f"   - ✅ Crops API v2.0")
print(f"   - ✅ Exports API v2.0")
print(f"   - ✅ Statistics API v2.0")
print("\n🔧 시작 방법:")
print("   1. 모든 셀 실행")
print("   2. start_server() 함수 호출")
print("   3. http://localhost:8000/api/docs 접속")
print("\n📚 문서:")
print("   - API 문서: /api/docs")
print("   - Redoc: /api/redoc")
print("   - 헬스 체크: /health")
print("\n🎯 다음 단계:")
print("   - JWT 인증 시스템 구현")
print("   - 통합 테스트 작성")
print("   - 프로덕션 배포")
print("="*60)

# 테스트 실행
test_apis()